# LSTM기반 문장 생성with전처리
- 예제 : 간단한 노래 가사로 학습

클래스로 만들어보기

In [9]:
!pip install konlpy

In [10]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense, Bidirectional, Dropout

In [11]:
text = '''그대 기억이 지난 사랑이

내 안을 파고드는 가시가 되어

제발 가라고 아주 가라고

애써도 나를 괴롭히는데

아픈 만큼 너를 잊게 된다면

차라리 앓고 나면 그만인데

가시처럼 깊게 박힌 기억은

아파도 아픈 줄 모르고

그대 기억이 지난 사랑이

내 안을 파고드는 가시가 되어

제발 가라고 아주 가라고'''

In [12]:
raw_data = text.split("\n\n")
raw_data

['그대 기억이 지난 사랑이',
 '내 안을 파고드는 가시가 되어',
 '제발 가라고 아주 가라고',
 '애써도 나를 괴롭히는데',
 '아픈 만큼 너를 잊게 된다면',
 '차라리 앓고 나면 그만인데',
 '가시처럼 깊게 박힌 기억은',
 '아파도 아픈 줄 모르고',
 '그대 기억이 지난 사랑이',
 '내 안을 파고드는 가시가 되어',
 '제발 가라고 아주 가라고']

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(raw_data)

In [14]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

32

In [15]:
print(tokenizer.word_index)

{'가라고': 1, '그대': 2, '기억이': 3, '지난': 4, '사랑이': 5, '내': 6, '안을': 7, '파고드는': 8, '가시가': 9, '되어': 10, '제발': 11, '아주': 12, '아픈': 13, '애써도': 14, '나를': 15, '괴롭히는데': 16, '만큼': 17, '너를': 18, '잊게': 19, '된다면': 20, '차라리': 21, '앓고': 22, '나면': 23, '그만인데': 24, '가시처럼': 25, '깊게': 26, '박힌': 27, '기억은': 28, '아파도': 29, '줄': 30, '모르고': 31}


## 훈련 데이터 생성
raw_data1 = '그대 기억이 지난 사랑이'
- x1 = '그대', y1 = '기억이'
- x2 = '그대 기억이', y2 = ' 지난'
- x3 = '그대 기억이 지난', y3 = '사랑이'

### Implementation 1
- 1단계 : 벡터화  
'그대 기억이 지난 사랑이'    
=>
[1, 2, 3, 4]   
=>   
[[1, 2],  
 [1, 2, 3],  
 [1, 2, 3, 4]]
  
- 2단계 : 패딩  
=>  
[[0, 0, 0, 1, 2],  
 [0, 0, 1, 2, 3],  
 [0, 1, 2, 3, 4]]
  
- 3단계 : x / y 분할   
=>  
x =[[0, 0, 0, 1],  
    [0, 0, 1, 2],  
    [0, 1, 2, 3]]  
y = [2,  
     3,  
     4]


In [16]:
# 1단계 : 벡터화
# '그대 기억이 지난 사랑이'
# => [1, 2, 3, 4]
# =>
# [[1, 2],
# [1, 2, 3],
# [1, 2, 3, 4]]

sequence_list = []
# 앞에 단어가 들어오면 뒤의 단어가 출력되어야 한다.
# 맨 뒤에 하나가 뒤의 단어(y), 그 앞에 단어들이 앞의 단어(x)

for line in raw_data: # 줄바꿈을 기준으로 문장나누기
    # tokenizer.texts_to_sequences
    encoded = tokenizer.texts_to_sequences([line])[0]
    for cnt in range(1, len(encoded)):
        sequence_list.append(encoded[:cnt + 1])

In [17]:
# 2단계 : 패딩 (pre)
# =>
MAX_SEQUENCE_LENGTH = 5
# tmp = []
# for seq in sequence_list:
#     tmp.append([0] * (padding - len(seq)) + seq)
# # [[0, 0, 0, 1, 2],
# # [0, 0, 1, 2, 3],
# # [0, 1, 2, 3, 4]]
# sequence_list = tmp

# pad_sequences 사용!!!
padded = pad_sequences(sequence_list, maxlen=MAX_SEQUENCE_LENGTH, padding='pre')

In [18]:
# 3단계 : origin -> x / y 분할 => x = :-1(마지막 토큰만 제외) / y = -1 (마지막 토큰만)
# =>
padded_array = np.array(padded)
X = padded_array[:, :-1]
y = padded_array[:, -1:].reshape(-1)
print(X)
# x =[[0, 0, 0, 1],
# [0, 0, 1, 2],
# [0, 1, 2, 3]]
# y = [2,
# 3,
# 4]


[[ 0  0  0  2]
 [ 0  0  2  3]
 [ 0  2  3  4]
 [ 0  0  0  6]
 [ 0  0  6  7]
 [ 0  6  7  8]
 [ 6  7  8  9]
 [ 0  0  0 11]
 [ 0  0 11  1]
 [ 0 11  1 12]
 [ 0  0  0 14]
 [ 0  0 14 15]
 [ 0  0  0 13]
 [ 0  0 13 17]
 [ 0 13 17 18]
 [13 17 18 19]
 [ 0  0  0 21]
 [ 0  0 21 22]
 [ 0 21 22 23]
 [ 0  0  0 25]
 [ 0  0 25 26]
 [ 0 25 26 27]
 [ 0  0  0 29]
 [ 0  0 29 13]
 [ 0 29 13 30]
 [ 0  0  0  2]
 [ 0  0  2  3]
 [ 0  2  3  4]
 [ 0  0  0  6]
 [ 0  0  6  7]
 [ 0  6  7  8]
 [ 6  7  8  9]
 [ 0  0  0 11]
 [ 0  0 11  1]
 [ 0 11  1 12]]


In [19]:
print('original:\n', padded[:3])
print('X\n', X[:3])
print('y\n', y[:3])

original:
 [[0 0 0 2 3]
 [0 0 2 3 4]
 [0 2 3 4 5]]
X
 [[0 0 0 2]
 [0 0 2 3]
 [0 2 3 4]]
y
 [3 4 5]


In [20]:
X.shape, y.shape

((35, 4), (35,))

In [21]:
y

array([ 3,  4,  5,  7,  8,  9, 10,  1, 12,  1, 15, 16, 17, 18, 19, 20, 22,
       23, 24, 26, 27, 28, 13, 30, 31,  3,  4,  5,  7,  8,  9, 10,  1, 12,
        1])

## 모델 구축

### Implementation 2

In [22]:
# Functional API

input = tf.keras.Input(shape=(4, ))

# embedding
x = Embedding(len(tokenizer.word_index) + 1, 32)(input)
# lstm
x = LSTM(128)(x)
# dense
output = Dense(32, activation = 'softmax')(x)
 
model = tf.keras.Model(inputs = input, outputs = output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 embedding (Embedding)       (None, 4, 32)             1024      
                                                                 
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
Total params: 87,584
Trainable params: 87,584
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer ='adam',
              metrics = ['acc'])

In [24]:
model.fit(X, y, epochs = 300, verbose = 2)

Epoch 1/300
2/2 - 2s - loss: 3.4648 - acc: 0.0857 - 2s/epoch - 1s/step
Epoch 2/300
2/2 - 0s - loss: 3.4583 - acc: 0.1714 - 9ms/epoch - 5ms/step
Epoch 3/300
2/2 - 0s - loss: 3.4537 - acc: 0.1143 - 8ms/epoch - 4ms/step
Epoch 4/300
2/2 - 0s - loss: 3.4490 - acc: 0.1714 - 7ms/epoch - 4ms/step
Epoch 5/300
2/2 - 0s - loss: 3.4448 - acc: 0.1714 - 8ms/epoch - 4ms/step
Epoch 6/300
2/2 - 0s - loss: 3.4404 - acc: 0.1714 - 9ms/epoch - 4ms/step
Epoch 7/300
2/2 - 0s - loss: 3.4358 - acc: 0.1714 - 7ms/epoch - 4ms/step
Epoch 8/300
2/2 - 0s - loss: 3.4312 - acc: 0.1714 - 7ms/epoch - 4ms/step
Epoch 9/300
2/2 - 0s - loss: 3.4260 - acc: 0.1714 - 7ms/epoch - 4ms/step
Epoch 10/300
2/2 - 0s - loss: 3.4201 - acc: 0.1714 - 6ms/epoch - 3ms/step
Epoch 11/300
2/2 - 0s - loss: 3.4133 - acc: 0.1714 - 28ms/epoch - 14ms/step
Epoch 12/300
2/2 - 0s - loss: 3.4056 - acc: 0.1714 - 9ms/epoch - 4ms/step
Epoch 13/300
2/2 - 0s - loss: 3.3959 - acc: 0.1143 - 7ms/epoch - 4ms/step
Epoch 14/300
2/2 - 0s - loss: 3.3853 - acc: 0.1

In [25]:
y_pred = model.predict(X[0:1])

1/1 [==============================] - 0s 398ms/step


In [26]:
# vocab들 중에 각 단어일 확률
np.argmax(y_pred)

3

In [27]:
tokenizer.index_word[3]

'기억이'

## 문장 생성 함수 구축

### Implementation 3

In [28]:
tokenizer.word_index.items()

dict_items([('가라고', 1), ('그대', 2), ('기억이', 3), ('지난', 4), ('사랑이', 5), ('내', 6), ('안을', 7), ('파고드는', 8), ('가시가', 9), ('되어', 10), ('제발', 11), ('아주', 12), ('아픈', 13), ('애써도', 14), ('나를', 15), ('괴롭히는데', 16), ('만큼', 17), ('너를', 18), ('잊게', 19), ('된다면', 20), ('차라리', 21), ('앓고', 22), ('나면', 23), ('그만인데', 24), ('가시처럼', 25), ('깊게', 26), ('박힌', 27), ('기억은', 28), ('아파도', 29), ('줄', 30), ('모르고', 31)])

In [29]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # 현재 단어에 대한 정수 인코딩과 패딩
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        padded = pad_sequences([encoded], maxlen=MAX_SEQUENCE_LENGTH, padding='pre')

        # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        # model.predict / np.argmax
        y_pred = model.predict(padded)
        result = tokenizer.index_word[np.argmax(y_pred)]

        for word, index in tokenizer.word_index.items():
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면 break
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word += ' ' + result
        # 예측 단어를 문장에 저장

    return current_word


In [30]:
print(sentence_generation(model, tokenizer, '제발', 4))

1/1 [==============================] - 0s 20ms/step
제발 가라고 아주 가라고 가라고


In [31]:
print(sentence_generation(model, tokenizer, '그냥', 5))

1/1 [==============================] - 0s 13ms/step
그냥 가라고 앓고 앓고 나면 나면


## NSMC 영화 리뷰 데이터로 수행

In [2]:
# 영화 리뷰 데이터
train = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', header=0, delimiter='\t' ,quoting=3)
test = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt', header=0, delimiter='\t' ,quoting=3)

In [3]:
train = train.dropna()

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['document'].values)
vocab_size = len(tokenizer.word_index) + 1

In [7]:
sequence_list = []
lenn = []
for value in train['document'].values:
    sequence_list.append(tokenizer.texts_to_sequences([value])[0])
    lenn.append(len(tokenizer.texts_to_sequences([value])[0]))

In [8]:
padded = pad_sequences(sequence_list, maxlen=10, padding='pre')

In [32]:
X = padded[:, :-1]
y = padded[:, -1:]

In [33]:
# Functional API

input = tf.keras.Input(shape=(9, ))

# embedding
x = Embedding(vocab_size, 100)(input)
# lstm
x = LSTM(128)(x)
# dense
output = Dense(vocab_size, activation = 'softmax')(x)
 
model = tf.keras.Model(inputs = input, outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding_1 (Embedding)     (None, 9, 100)            3200      
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 32)                4128      
                                                                 
Total params: 124,576
Trainable params: 124,576
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer ='adam',
              metrics = ['acc'])

## 뉴스 기사 데이터로 수행

In [ ]:
# 뉴스 기사 데이터
!pip install -U --no-cache-dir gdown --pre
!gdown --no-cookies --id 1KuBG40WNpVPV1ilfGaiCI2D3l8JVSfbS